20190325 | [ [DL2v3L8@1:22:42](https://youtu.be/4u8FxNEDUeg?t=4962) ]

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

We want a mean,stdev of (0,1). Use the training-set's normalization for all.

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [7]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

The model is going to have 1 hidden/internal layer. Normally we want the output to have 10 activations bc we'd use cross-entropy against those 10 actvns; but to simplify things for now: we're going to use MSE instead of CE -- so we'll have 1 activation. 

### Basic architecture

In [69]:
# num hidden
nh = 50

for layers we need 2 weight matrices and 2 bias vectors

In [70]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [71]:
w1.mean(), w1.std()

(tensor(-0.0002), tensor(0.0357))

the input to layer-1 are normlzd to mean,stdv (0,1); we also need inputs to layer-2 to be (0,1).

**Notice**: above, if you *don't* divide by the √ of the number of rows of the weight-matrix, you'll get a mess of mean,stdv output. If you *do*, you'll get an output with a (μ,σ) *very* close to (0,1).

---

[DL2v3L8@1:28:05](https://youtu.be/4u8FxNEDUeg?t=5285) | in general, normal random numbers of mean-0 and stdev of 1 / √(num_rows) will give an output of ~(0,1). This is *critically* important for training neural networks. In the Fixup Initialization paper, a 10,000-layer deep neural network was trained *with no normalization layers* by just doing careful initialization. 

[ [thoughts](https://forums.fast.ai/t/chit-chat-thread/41345/121?u=borz) ]

In [72]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [73]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

our simple affine function from before:

In [74]:
def lin(x, w, b): return x@w + b

In [75]:
t = lin(x_valid, w1, b1)

this cell will have a large (μ,σ) if `/math.sqrt(m)` wasn't used; near (0,1) if used.

In [76]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0276), tensor(0.9448))

`t = lin(x_valid, w1, b1)` is not actually how the first layer is defined. The first layer has an output nonlinearity function: a zero-max (ReLU).

There're a lot of ways to do a zero-max, but a single function in Pytorch if one exists is almost always the fastest way to do it, since it's usually written in C.

In [77]:
def relu(x): return x.clamp_min(0.)

In [78]:
t = relu(lin(x_valid, w1, b1))

In [79]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3868), tensor(0.5547))

This doesn't have a μ,σ of (0,1) because we took values with mean-0 stdev-1, and removed all values less than zero (set them to 0). The result no longer has a mean of zero, and has about half the stdev it used to.

---

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

---

The way we account for a zero-max is to put a 2 on the top instead of a 1:

In [80]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [81]:
w1.mean(),w1.std()

(tensor(0.0003), tensor(0.0505))

In [82]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.4960), tensor(0.7617))

This result (0.4960, .7617) may not be great, but it is (supposedly) better than the previous (0.3868, 0.5547).

Kaiming Initialization is essentially √(2 / nl) where nl : num_activations.

This gives us a better variance, but the mean is still not good. The reason is that we essentially deleted everything under zero, so the mean is now half, not zero. *So*.... maybe just subtract by **0.5**? That's exactly what you do.

In [93]:
#export
from torch.nn import init

`'fan_out'`: divide by num_cols; `'fan_in'`: divide by num_rows. 'fan_out' preserves the unit variance in the backward pass, 'fan_in': the forward pass. (ie: keep variance = 1).

For: 
```
w1 = torch.randn(m,nh)/math.sqrt(m)
```
'fan_out' will use `m`; 'fan_in' will use `nh`.

Why do we use 'fan_out' when we're looking for the forward pass (ie: 'fan_out' *should* be dividing by `nh`, not `m`)?

The reason is in PyTorch a linear layer doesn't just do a matmul, *it does a matmul with a transpose*. So `torch.nn.Linear(50, 784).weight.shape` returns `torch.Size([784, 50])`. This means it'll turn [50, 784] into [784, 50] (transpose) and *then* do a matmul.

*This is why* we need to supply the opposite information, by telling it to use 'fan_out', because `torch.nn.Linear` transposes the matrix before multiplying, so the rows and columns get switched.

So we'd use  'fan_in' with our manual layer initialization, but using PyTorch's: we use 'fan_out' since it does a transpose (thus swapping axes).

In [100]:
torch.nn.Linear(50, 784).weight.shape

torch.Size([784, 50])

In [94]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [89]:
init.kaiming_normal_??

In [95]:
w1.mean(),w1.std()

(tensor(-0.0002), tensor(0.0504))

In [96]:
t.mean(),t.std()

(tensor(0.5216), tensor(0.7767))

In [97]:
w1.shape

torch.Size([784, 50])

In [98]:
import torch.nn

In [101]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

To investigate why 'fan_out' is used instead of 'fan_in', we can look at how PyTorch initializes its Linear layers. `torch.nn.Linear.forward` calls `F.linear` which is `torch.nn.functional.linear`, so we check that (and that's where we find that PyTorch does transposes with matmuls via: 

    output = input.matmul(weight.t())

In [102]:
torch.nn.Linear.forward??

In [114]:
torch.nn.functional.linear??

We see how PyTorch initializes linear layers; what about convolutional?

In [116]:
torch.nn.Conv2d??

`torch.nn.Cond2d` passes all its code down to `_ConvNd` which is located in:

    File:           ~/Miniconda3/envs/fastai/lib/python3.7/site-packages/torch/nn/modules/conv.py
    
So we check `torch.nn.modules.conv._ConvNd`:

In [113]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [112]:
init.kaiming_uniform_??

`torch.nn.modules.conv._ConvNd` initializes with `kaiming_uniform_` with a multiplier `a=math.sqrt(5)`.

    a: the negative slope of the rectifier used after this layer (0 for ReLU by default)
    
It's currently (20190324) unknown why it's √5, and it tends to work poorly. 

---

Back to handling the mean issue from kaiming-init: subtract 0.5 from the zero-max:

In [117]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [118]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0617), tensor(0.8207))

Now the mean is closer to zero, and the stdev closer to one.

---

***Aside***: why above is:

```
# simplified kaiming init
w1 = torch.randn(m,nh)/math.sqrt(m)
```
but below that:
```
# kaiming init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m)
```

In [125]:
trand = torch.randn(m,nh)

In [135]:
trand.mean(), trand.std()

(tensor(0.0087), tensor(0.9977))

In [127]:
wtmp_smp = trand / math.sqrt(m)
wtmp_rlu =trand * math.sqrt(2./m)

In [128]:
wtmp_smp.mean(), wtmp_smp.std()

(tensor(0.0003), tensor(0.0356))

In [129]:
wtmp_rlu.mean(), wtmp_rlu.std()

(tensor(0.0004), tensor(0.0504))

In [130]:
math.sqrt(m)

28.0

In [131]:
math.sqrt(2./m)

0.050507627227610534

In [132]:
1 / 28, 1 * (math.sqrt(2/m))

(0.03571428571428571, 0.050507627227610534)

That's why. You're making the tensor smaller --> so for `math.sqrt(m)`, `m` is 784, the number of input values. So you *divide* by √784 = **28**. √(2 / 784) ≈ **0.0505**, so you *multiply* by it.

The reason why we're initializing (essent. renormalizing) weight matrix `w1` is so that *its activations have a mean,stdev of `(0,1)`*. The input to W1 is N(0,1). We need to ensure W1's output -- the input to W2 -- is *also* N(0,1). That's the purpose of layer initialization: to ensure all neural network layers are working on N(0,1) values.

N(0,1) : normal distribution w/ mean,stdev = (0,1).

***End Aside***

---

At this point, we have ReLU & Init and we have Linear so we can do a forward pass. Here's the model: a linear layer, a relu layer, and another linear layer.

In [138]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [139]:
%timeit -n 10 _=model(x_valid)

12.4 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [140]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [141]:
model(x_valid).shape

torch.Size([10000, 1])

Mean Squared Error expects something of shape (10000,) not (10000, 1). In PyTorch, use `unsqueeze` to insert a unit axis, and `squeeze` to remove.

---

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [143]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [144]:
y_train,y_valid = y_train.float(),y_valid.float()

In [145]:
preds = model(x_train)

In [146]:
preds.shape

torch.Size([50000, 1])

In [147]:
mse(preds, y_train)

tensor(37.9455)

### Gradients and backward pass

The chain-rule calculation being done in the backward pass calculates the gradient of the forward pass:

    mse( linear2( relu( linear1( x ) ) ) )
    
wrt. the loss. To implement chain-rule differentiation manually, start with the last (outermost) function: the loss-fn MSE:

```
MSE = (input - target)**2 --> Δ(MSE) = 2*(input - target)
```

---

Each stage of the chain-rule is multiplied. So the gradient is stored in the `.g` attribute of the previous layer's output, `inp`. The input `inp` of `mse_grad` is the output `out` of the previous layer. Ie: `inp` in `mse_grad` is `out` in `lin_grad`. So the gradient can be accessed by calling the `.g` attrib. of `out` in the next stage.

In [148]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

The gradient of a ReLU is zero where zero, and one where greater.

In [149]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

The gradient of a matprod is the matprod with its transpose.

In [150]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()  # grad of output wrt. inputs
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0) # grad of output wrt. weights
    b.g = out.g.sum(0) # grad of output wrt. biases

Backpropagation is just the chain-rule, with all intermediate calculations stored. At each stage of the backward pass we pass in that stage's forward pass result and the gradient of the next layer.

***Note*** *that the `loss` is never actually (directly) used in gradient calculations.*

In [151]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

Perform a single forward & backward pass on the training set:

In [152]:
forward_and_backward(x_train, y_train)

Here we store the gradients for use in an optimizer.

In [153]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

---

[ [L8@1:56:54](https://youtu.be/4u8FxNEDUeg?t=7014) ] `requires_grad_` is how you take a PyTorch tensor and turn it into an auto-grad'ified PyTorch tensor. PyTorch will keep track of every computation done on the tensor and compute the gradient.

In [154]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [155]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [156]:
loss = forward(xt2, y_train)

In [157]:
loss.backward()

PyTorch stores gradients in `.grad`. not `.g`.

In [158]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model [ [L8@1:68:34](https://youtu.be/4u8FxNEDUeg?t=7114) ]

### Layers as classes

In [159]:
class Relu():
    def __call__(self, inp): # use __call__ for forward
        self.inp = inp  # save input
        self.out = inp.clamp_min(0.)-0.5 # save output
        return self.out # return output
    # save gradient in input's `.g`
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [160]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [161]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp   # save input
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean() # forward pass & save output
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [162]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

Set all gradients to None so we know we're not cheating, and init a model:

In [163]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

`__call__` lets us call `model` as if it were a function

In [164]:
%time loss = model(x_train, y_train)

CPU times: user 161 ms, sys: 15.4 ms, total: 176 ms
Wall time: 92.7 ms


In [165]:
%time model.backward()

CPU times: user 9.1 s, sys: 1min, total: 1min 9s
Wall time: 4min 49s


In [166]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

More refactoring to remove duplicate code

In [167]:
class Module():
    def __call__(self, *args):
        self.args = args # self.inp = inp
        self.out = self.forward(*args) # call forward
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [168]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [169]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        # replace the large outer-prod-&-sum w/ einsum
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [170]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [171]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [172]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [173]:
%time loss = model(x_train, y_train)

CPU times: user 167 ms, sys: 307 ms, total: 474 ms
Wall time: 266 ms


In [174]:
%time model.backward()

CPU times: user 296 ms, sys: 95.4 ms, total: 391 ms
Wall time: 271 ms


In [175]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum [ [L8@2:04:44](https://youtu.be/4u8FxNEDUeg?t=7484) ]

In [176]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g # einsum is eqvt to this
        self.b.g = out.g.sum(0)

In [177]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [178]:
%time loss = model(x_train, y_train)

CPU times: user 134 ms, sys: 3.66 ms, total: 138 ms
Wall time: 70.8 ms


In [179]:
%time model.backward()

CPU times: user 286 ms, sys: 68.2 ms, total: 354 ms
Wall time: 220 ms


In [180]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

We implemented `torch.nn.Linear` and `torch.nn.Module`, so from our rules we can start using them.

### nn.Linear and nn.Module

In [181]:
#export
from torch import nn

In [182]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [183]:
model = Model(m, nh, 1)

In [184]:
%time loss = model(x_train, y_train)

CPU times: user 138 ms, sys: 5.92 ms, total: 144 ms
Wall time: 89.5 ms


In [185]:
%time loss.backward()

CPU times: user 132 ms, sys: 8.25 ms, total: 140 ms
Wall time: 83.6 ms


The PyTorch forward pass takes the same time as ours, but their backward pass is about twice as fast. Likely due to optimizations such as only calculating needed gradients (we're calc. all).

## Export

In [1]:
!./notebook2script.py 02_fully_connected-Copy1.ipynb

Converted 02_fully_connected-Copy1.ipynb to nb_02.py
